In [37]:
import pickle
import numpy as np
import pandas as pd

In [38]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [39]:
test

,label,x,y,speed,strokes,angles
382,0,"[513.0, 469.0, 429.0, 408.0, 388.0, 373.0, 364...","[105.0, 139.0, 187.0, 226.0, 268.0, 314.0, 360...",1.715,1,"[2.48, 2.27, 2.06, 2.02, 1.89, 1.76, 1.56, 1.3..."
892,6,"[936.0, 931.0, 907.0, 831.0, 809.0, 791.0, 771...","[12.0, 8.0, 11.0, 30.0, 38.0, 46.0, 60.0, 76.0...",0.476,1,"[-2.47, 3.02, 2.9, 2.79, 2.72, 2.53, 2.47, 2.3..."
123,6,"[786.0, 760.0, 725.0, 709.0, 650.0, 610.0, 571...","[154.0, 152.0, 156.0, 159.0, 175.0, 191.0, 211...",1.279,1,"[-3.06, 3.03, 2.96, 2.88, 2.76, 2.67, 2.44, 2...."
499,0,"[690.0, 653.0, 622.0, 586.0, 546.0, 506.0, 486...","[106.0, 116.0, 130.0, 149.0, 174.0, 205.0, 222...",1.279,1,"[2.88, 2.72, 2.66, 2.58, 2.48, 2.44, 2.29, 2.1..."
212,6,"[613.0, 602.0, 588.0, 570.0, 531.0, 518.0, 478...","[230.0, 231.0, 240.0, 255.0, 289.0, 302.0, 344...",1.560,1,"[3.05, 2.57, 2.45, 2.42, 2.36, 2.33, 2.28, 2.2..."
...,...,...,...,...,...,...
839,8,"[846.0, 842.0, 818.0, 744.0, 768.0, 804.0, 829...","[304.0, 285.0, 281.0, 330.0, 349.0, 369.0, 395...",1.085,1,"[-1.78, -2.98, 2.56, 0.67, 0.51, 0.81, 1.27, 1..."
273,5,"[801.0, 796.0, 793.0, 779.0, 767.0, 752.0, 735...","[192.0, 192.0, 193.0, 195.0, 196.0, 197.0, 199...",Infin,1,"[3.14, 2.82, 3.0, 3.06, 3.08, 3.02, 3.03, 3.08..."
390,8,"[709.0, 690.0, 685.0, 675.0, 674.0, 678.0, 692...","[204.0, 214.0, 219.0, 246.0, 267.0, 287.0, 307...",2.285,1,"[2.66, 2.36, 1.93, 1.62, 1.37, 0.96, 0.62, 0.4..."
310,1,"[676.0, 676.0, 676.0, 676.0, 674.0, 672.0, 669...","[159.0, 179.0, 198.0, 228.0, 250.0, 275.0, 300...",2.133,1,"[1.57, 1.57, 1.57, 1.66, 1.65, 1.69, 1.61, 1.6..."


In [40]:
predicted_test = test.copy()

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

log_likelihoods = pd.DataFrame()
log_likelihoods['predicted_class'] = classes
log_like_max = []
top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
is_predicted = []

for i in range(len(test)):

    true_label = test['label'].iloc[i]
    list_log_likelihoods = []
    for model in models:
        
        angle = test['angles'].iloc[i]
        X = [[one] for one in angle]
        list_log_likelihoods.append(model.score(X))
        pass

    log_likelihoods['log_likelihood'] = list_log_likelihoods
    log_likelihoods['true_class'] = true_label

    # Compute top 1 prediction
    max_log_likelihood = log_likelihoods.loc[log_likelihoods['log_likelihood'].idxmax()]
    log_like_max.append(max_log_likelihood['log_likelihood'])
    predicted_class.append(max_log_likelihood['predicted_class'])

    if max_log_likelihood['predicted_class'] == max_log_likelihood['true_class']:
        is_predicted.append(True)
    else:
        is_predicted.append(False)

    # Compute top 3 predictions
    top_3_loglikelihood.append(sorted(list_log_likelihoods, reverse=True)[:3])
    top1, top2, top3 = sorted(list_log_likelihoods, reverse=True)[:3]
    top_3_list = []
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top1]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top2]['predicted_class'].values[0])
    top_3_list.append(log_likelihoods.loc[log_likelihoods['log_likelihood'] == top3]['predicted_class'].values[0])
    top_3_predicted_classes.append(top_3_list)




predicted_test["predicted_class"] = predicted_class
predicted_test["log_likelihood"] = log_like_max
predicted_test["is_predicted"] = is_predicted
predicted_test["top_3_predicted"] = top_3_predicted_classes
predicted_test["top_3_loglikelihood"] = top_3_loglikelihood

predicted_test

# model_0.score_samples(angles_0)

,label,x,y,speed,strokes,angles,predicted_class,log_likelihood,is_predicted,top_3_predicted,top_3_loglikelihood
382,0,"[513.0, 469.0, 429.0, 408.0, 388.0, 373.0, 364...","[105.0, 139.0, 187.0, 226.0, 268.0, 314.0, 360...",1.715,1,"[2.48, 2.27, 2.06, 2.02, 1.89, 1.76, 1.56, 1.3...",0.0,-20.791278,True,"[0, 6, 8]","[-20.79127790353417, -25.204031861634927, -28...."
892,6,"[936.0, 931.0, 907.0, 831.0, 809.0, 791.0, 771...","[12.0, 8.0, 11.0, 30.0, 38.0, 46.0, 60.0, 76.0...",0.476,1,"[-2.47, 3.02, 2.9, 2.79, 2.72, 2.53, 2.47, 2.3...",0.0,-45.728407,False,"[0, 6, 8]","[-45.72840676998645, -46.727649511602515, -65...."
123,6,"[786.0, 760.0, 725.0, 709.0, 650.0, 610.0, 571...","[154.0, 152.0, 156.0, 159.0, 175.0, 191.0, 211...",1.279,1,"[-3.06, 3.03, 2.96, 2.88, 2.76, 2.67, 2.44, 2....",6.0,-14.901149,True,"[6, 5, 9]","[-14.901148891278469, -26.120476856778186, -26..."
499,0,"[690.0, 653.0, 622.0, 586.0, 546.0, 506.0, 486...","[106.0, 116.0, 130.0, 149.0, 174.0, 205.0, 222...",1.279,1,"[2.88, 2.72, 2.66, 2.58, 2.48, 2.44, 2.29, 2.1...",0.0,-26.005333,True,"[0, 6, 8]","[-26.005333416618136, -28.11712550895118, -36...."
212,6,"[613.0, 602.0, 588.0, 570.0, 531.0, 518.0, 478...","[230.0, 231.0, 240.0, 255.0, 289.0, 302.0, 344...",1.560,1,"[3.05, 2.57, 2.45, 2.42, 2.36, 2.33, 2.28, 2.2...",6.0,-14.954131,True,"[6, 0, 9]","[-14.954130909394893, -21.64260147784129, -25...."
...,...,...,...,...,...,...,...,...,...,...,...
839,8,"[846.0, 842.0, 818.0, 744.0, 768.0, 804.0, 829...","[304.0, 285.0, 281.0, 330.0, 349.0, 369.0, 395...",1.085,1,"[-1.78, -2.98, 2.56, 0.67, 0.51, 0.81, 1.27, 1...",8.0,-28.372513,True,"[8, 0, 9]","[-28.372513297247355, -44.2521539318253, -46.3..."
273,5,"[801.0, 796.0, 793.0, 779.0, 767.0, 752.0, 735...","[192.0, 192.0, 193.0, 195.0, 196.0, 197.0, 199...",Infin,1,"[3.14, 2.82, 3.0, 3.06, 3.08, 3.02, 3.03, 3.08...",5.0,-23.783729,True,"[5, 3, 9]","[-23.783729258170325, -25.895176294417478, -39..."
390,8,"[709.0, 690.0, 685.0, 675.0, 674.0, 678.0, 692...","[204.0, 214.0, 219.0, 246.0, 267.0, 287.0, 307...",2.285,1,"[2.66, 2.36, 1.93, 1.62, 1.37, 0.96, 0.62, 0.4...",8.0,-24.641497,True,"[8, 2, 0]","[-24.64149727197631, -36.32703188661775, -37.0..."
310,1,"[676.0, 676.0, 676.0, 676.0, 674.0, 672.0, 669...","[159.0, 179.0, 198.0, 228.0, 250.0, 275.0, 300...",2.133,1,"[1.57, 1.57, 1.57, 1.66, 1.65, 1.69, 1.61, 1.6...",1.0,22.281200,True,"[1, 5, 0]","[22.281200178638716, -2.109317967548722, -6.32..."


In [41]:
# Calculate top 3 accuracy
sum_top_3 = 0
for idx, row in predicted_test.iterrows():
    for three_likelihood in row["top_3_predicted"]:
        if(row["label"] == three_likelihood):
            sum_top_3 += 1
            pass

accuracy = sum(predicted_test["is_predicted"])/len(predicted_test["is_predicted"])
print("Accuracy: ", accuracy)
accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy:  0.875
Accuracy top 3:  0.984375
